In [ ]:
from selenium import webdriver
import pandas as pd
import time


def enter_address(driver, address):
    """
    Enters address into search text field and gets corresponding latitude and longitude.
    Write to text file.
    """
    actions = webdriver.ActionChains(driver)
    #finds the text field to enter address in
    search_field = driver.find_element_by_xpath("//*[@id='latlongform']")
    actions.move_to_element(search_field)
    actions.click()
    #types address in
    actions.send_keys(address + " BOSTON MA")
    actions.perform()
    #clicks enter
    search_field.submit()
    time.sleep(3)
    #finds coordinates at bottom of page
    coord_field = driver.find_element_by_class_name("coordinatetxt")
    coords = coord_field.text

    #appends coords to locations file
    with open("fio_locations.txt", "a") as file:
        format_coords = coords[1:-1].split(",")
        file.write(format_coords[0] + "," + format_coords[1][1:] + "\n")

    #clears search text field for next address
    text_field = driver.find_element_by_xpath("//*[@id='gadres']")
    text_field.clear()
    

def scrape_lat_long(url, driver, address_list):
    """
    Scrapes latlong.net for latitude and longitude of a list of addresses.
    """
    counter = 0

    for address in address_list:
        try:
            enter_address(driver, address)
            counter += 1
        except:   #means that the address was not valid
            print(counter)
            print(address)
            driver.close()
            driver = webdriver.Chrome('/Users/elisa/Desktop/chromedriver')
            driver.get(url)
            if("at" in address):   #common occurrence at intersections, just take one of the streets and try again
                address = address.split("at")[1   ]
            else:                 #if that wasn't the case, just a bad address
                with open("fio_locations.txt", "a") as file:
                    file.write("bad address\n")
                counter += 1
                continue
            try:                 #try again without the "at"
                enter_address(driver,address)
                counter += 1
            except:            #if it still didn't work, bad address
                print(counter)
                print(address)
                driver.close()
                driver = webdriver.Chrome('/Users/elisa/Desktop/chromedriver')
                driver.get(url)
                with open("fio_locations.txt", "a") as file:
                    file.write("bad address\n")
                counter += 1
                continue
                                
    return coords_list
    
csv_path = "ACLUProject/datasets/bpd-field-interrogation-and-observation-reports.csv"
fio_df = pd.read_csv(csv_path)
address_list = list(fio_df["LOCATION"])
driver = webdriver.Chrome('/Users/elisa/Desktop/chromedriver')  #change this with the path on your machine
url = "http://www.latlong.net/"
driver.get(url)
coords_list = scrape_lat_long(url, driver, address_list)



